In [1]:
!pip install h2o

  Created wheel for h2o: filename=h2o-3.32.1.3-py2.py3-none-any.whl size=164854343 sha256=c30f0e02d7113fa321def42b0078fc0e7ec6f8451a5b1dae812bf9964f6d576d
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\72\00\18\d1ed0b56eb5efd5e96b48828c07bd131ff8829a6d16fcef39d
Successfully built h2o


In [2]:
import h2o
import pandas as pd

In [6]:

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.291-b10, mixed mode)
  Starting server from C:\Users\Administrator\anaconda3\envs\mlp\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ADMINI~1\AppData\Local\Temp\tmp_tuj6pms
  JVM stdout: C:\Users\ADMINI~1\AppData\Local\Temp\tmp_tuj6pms\h2o_Administrator_started_from_python.out
  JVM stderr: C:\Users\ADMINI~1\AppData\Local\Temp\tmp_tuj6pms\h2o_Administrator_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_Administrator_89pgq9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.427 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
data = pd.read_csv('data_v1.csv')
data

,status,employee_count_1hot_100,employee_count_1hot_1000,employee_count_1hot_10000,total_funding_1hot_100,total_funding_1hot_1000,total_funding_1hot_10000,total_funding_1hot_l_10000
0,1,0,1,0,0,0,1,0
1,1,1,0,0,0,0,0,1
2,1,1,0,0,0,0,1,0
3,1,1,0,0,0,0,1,0
4,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
5504,1,0,0,1,0,0,0,1
5505,1,0,0,1,1,0,0,0
5506,1,0,0,1,1,0,0,0
5507,1,0,0,1,0,1,0,0


In [10]:
!pip install -U scikit-learn

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data, test_size = 0.3)
X_train_hex = h2o.H2OFrame(X_train)
X_test_hex = h2o.H2OFrame(X_test)
X_names =  ['employee_count_1hot_100', 'employee_count_1hot_1000', 'employee_count_1hot_10000', 'total_funding_1hot_100','total_funding_1hot_1000', 'total_funding_1hot_10000', 'total_funding_1hot_l_10000']
X_test = X_test_hex.drop('status').as_data_frame()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
# Define model
# h2o_rf = H2ORandomForestEstimator(ntrees=200, max_depth=20, nfolds=10)
# Train model
from h2o.automl import H2OAutoML
aml = H2OAutoML(
    max_runtime_secs=(3600 * 6),  # 8 hours
    max_models=None,  # no limit
    seed=20
)
aml.train(x=X_names, y='status', training_frame=X_train_hex)


AutoML progress: |
02:01:33.255: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_8db1168b850dc7775279f98f73ea48b2> was cancelled by the user.

In [ ]:

lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
out_path = "."

for m_id in model_ids:
    mdl = h2o.get_model(m_id)
    h2o.save_model(model=mdl, path=out_path, force=True)

h2o.export_file(lb, os.path.join(out_path, 'aml_leaderboard.h2o'), force=True)

In [ ]:
class H2OProbWrapper:
    def __init__(self, h2o_model, feature_names):
        self.h2o_model = h2o_model
        self.feature_names = feature_names
def predict_binary_prob(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        self.dataframe= pd.DataFrame(X, columns=self.feature_names)
        self.predictions = self.h2o_model.predict(h2o.H2OFrame(self.dataframe)).as_data_frame().values
        return self.predictions.astype('float64')[:,-1] 